In [ ]:
from pygeosys.geosys import Geosys
import os
from dotenv import load_dotenv
import datetime as dt
from dateutil.relativedelta import relativedelta
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

# read .env file
load_dotenv()

API_CLIENT_ID = os.getenv('API_CLIENT_ID')
API_CLIENT_SECRET = os.getenv('API_CLIENT_SECRET')
API_USERNAME = os.getenv('API_USERNAME')
API_PASSWORD = os.getenv('API_PASSWORD')

client = Geosys(API_CLIENT_ID, API_CLIENT_SECRET, API_USERNAME, API_PASSWORD)

In [ ]:
polygon = "POLYGON((-91.17523978603823 40.29787117039518,-91.17577285022956 40.29199489606421,-91.167613719932 40.29199489606421,-91.1673028670095 40.29867040193312,-91.17523978603823 40.29787117039518))"

today = dt.date.today()
year_ago = dt.date.today() + relativedelta(months=-12)

client.get_time_series(polygon, year_ago, today, "NDVI")


In [ ]:
client.get_time_series_by_pixel(polygon, year_ago, today, "NDVI")